In [1]:
from result_saver import SaverProvider, metadata_loader
provider = SaverProvider()

In [2]:
df = metadata_loader()

for job_id in df.job_id:
    provider.retrieve_job(job_id)

Custom Warning: Job ID cmx8c08atcpg0084s300 not found in /Users/mha/My Drive/Desktop/Studium/Physik/Master/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider...

Custom Warning: Job ID cmx8c08atcpg0084s300 is in an unknown state. Updating Metadata & Aborting...

Custom Warning: Job ID cmx8bzqwjay0008cnb90 not found in /Users/mha/My Drive/Desktop/Studium/Physik/Master/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider...

Custom Warning: Job ID cmx8bzqwjay0008cnb90 is in an unknown state. Updating Metadata & Aborting...

Custom Warning: Job ID cmx63npatcpg0084rx7g not found in /Users/mha/My Drive/Desktop/Studium/Physik/Master/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider...

Custom Warning: Job ID cmx63npatcpg0084rx7g is still running. Aborting...

Custom Warning: Job ID cmx63mewjay0008cn5s0 not found in /Users/mha/My Drive/Desktop/Studium/Physik/Master/Semester 3/IBM/IBM GIT/Soft-